<a href="https://colab.research.google.com/github/DionKimmm/2019_Summer_DL_Prof_Seok/blob/master/190729_RNN_BasicRNNCell.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 단계

## 필요한 라이브러리 불러오기
* Tensorflow, Numpy 등
## 데이터 불러오기
* Training, Test, Label data
## RNN 신경망 모델 구성
* 미리 계획된 구조대로 신경망 구성
## 최적화
## 훈련데이터를 이용한 학습
* Session을 열고, Epoch 수를 결정
* Epoch를 반복할 때 마다, error 값을 출력하여 error가 줄어들며 학습이 제대로 되는지 확인
* 데이터 사이즈가 클 경우, Batch를 통해 일정 사이즈의 데이터 만큼씩 학습을 수행
## 테스트 데이터를 이용한 최종 식별 결과 확인
* 모델로 예측한 값과 실제 레이블인 Y의 값을 비교
* 예측한 값에서 가장 큰 값을 예측한 레이블이라고 평가

### MNIST는 너무 질렸으니까 Fashion-MNIST를 써보자

In [1]:
!git clone https://github.com/zalandoresearch/fashion-mnist.git

Cloning into 'fashion-mnist'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 685 (delta 0), reused 0 (delta 0), pack-reused 681
Receiving objects: 100% (685/685), 105.20 MiB | 13.35 MiB/s, done.
Resolving deltas: 100% (393/393), done.


### 필요한 라이브러리 불러오기

In [0]:
import tensorflow as tf
import numpy as np


### 학습, 테스트, 레이블 데이터 불러오기


In [5]:
from tensorflow.examples.tutorials.mnist import input_data
fashion_mnist = input_data.read_data_sets('./fashion-mnist/data/fashion', one_hot=True)

Extracting ./fashion-mnist/data/fashion/train-images-idx3-ubyte.gz
Extracting ./fashion-mnist/data/fashion/train-labels-idx1-ubyte.gz
Extracting ./fashion-mnist/data/fashion/t10k-images-idx3-ubyte.gz
Extracting ./fashion-mnist/data/fashion/t10k-labels-idx1-ubyte.gz


In [0]:
# 옵션 설정
learning_rate = 0.001
total_epoch = 30
batch_size = 128

# RNN 은 순서가 있는 자료를 다루므로, 한 번에 입력받는 갯수와, 총 몇 단계로 
# 이루어져있는 데이터를 받을지를 설정
# 가로 픽셀수를 n_input 으로, 세로 픽셀수를 입력 단계인 n_step 으로 설정
n_input = 28
n_step = 28
n_hidden = 128
n_class = 10


### RNN 신경망 모델 구성

In [10]:
X = tf.placeholder(tf.float32, [None, n_step, n_input])  #n_step 이 추가 (CNN과 차이점)
Y = tf.placeholder(tf.float32, [None, n_class])

# RNN 에 학습에 사용할 셀을 생성합니다
# 다음 함수들을 사용하면 다른 구조의 셀로 간단하게 변경할 수 있습니다
cell = tf.nn.rnn_cell.BasicRNNCell(n_hidden)

# 다음처럼 tf.nn.dynamic_rnn 함수를 사용하면, CNN 의 tf.nn.conv2d 함수처럼 간단하게 RNN 신경망을 만듬 
outputs, states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)

# 결과를 Y의 다음 형식과 바꿔야 하기 때문에
# Y : [batch_size, n_class]
# outputs 의 형태를 이에 맞춰 변경해야합니다.
# outputs :  [batch_size, n_step, n_hidden]
#           ->  [n_step, batch_size, n_hidden]
#           ->  [batch_size, n_hidden]
outputs = tf.transpose(outputs, [1, 0, 2])
outputs = outputs[-1]

model = tf.layers.dense(outputs, 10)


W0729 05:43:50.940792 140325789505408 deprecation.py:323] From <ipython-input-10-77bb2ecc6258>:9: dynamic_rnn (from tensorflow.python.ops.rnn) is deprecated and will be removed in a future version.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
W0729 05:43:51.038969 140325789505408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0729 05:43:51.052680 140325789505408 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/rnn_cell_impl.py:459: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call init

### 손실함수 및 최적화

In [11]:
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=model, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(cost)

W0729 05:44:11.725591 140325789505408 deprecation.py:323] From <ipython-input-11-63c40467412b>:1: softmax_cross_entropy_with_logits (from tensorflow.python.ops.nn_ops) is deprecated and will be removed in a future version.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



### 훈련 데이터를 신경망 모델 학습

In [13]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

total_batch = int(fashion_mnist.train.num_examples/batch_size)
for epoch in range(total_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = fashion_mnist.train.next_batch(batch_size)

        # X 데이터를 RNN 입력 데이터에 맞게 [batch_size, n_step, n_input] 형태로 변환합니다.
        batch_xs = batch_xs.reshape((batch_size, n_step, n_input))

        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs, Y: batch_ys})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1), 'Avg. cost =', '{:.3f}'.format(total_cost / total_batch))
print('최적화 완료!')


Epoch: 0001 Avg. cost = 0.759
Epoch: 0002 Avg. cost = 0.527
Epoch: 0003 Avg. cost = 0.482
Epoch: 0004 Avg. cost = 0.451
Epoch: 0005 Avg. cost = 0.428
Epoch: 0006 Avg. cost = 0.415
Epoch: 0007 Avg. cost = 0.401
Epoch: 0008 Avg. cost = 0.395
Epoch: 0009 Avg. cost = 0.387
Epoch: 0010 Avg. cost = 0.371
Epoch: 0011 Avg. cost = 0.370
Epoch: 0012 Avg. cost = 0.359
Epoch: 0013 Avg. cost = 0.351
Epoch: 0014 Avg. cost = 0.355
Epoch: 0015 Avg. cost = 0.341
Epoch: 0016 Avg. cost = 0.340
Epoch: 0017 Avg. cost = 0.336
Epoch: 0018 Avg. cost = 0.335
Epoch: 0019 Avg. cost = 0.330
Epoch: 0020 Avg. cost = 0.324
Epoch: 0021 Avg. cost = 0.321
Epoch: 0022 Avg. cost = 0.322
Epoch: 0023 Avg. cost = 0.317
Epoch: 0024 Avg. cost = 0.310
Epoch: 0025 Avg. cost = 0.309
Epoch: 0026 Avg. cost = 0.307
Epoch: 0027 Avg. cost = 0.310
Epoch: 0028 Avg. cost = 0.299
Epoch: 0029 Avg. cost = 0.300
Epoch: 0030 Avg. cost = 0.294
최적화 완료!


### 테스트 데이터를 이용한 최종 식별 결과 확인

In [17]:
is_correct = tf.equal(tf.argmax(model, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))

test_batch_size = len(fashion_mnist.test.images)
test_xs = fashion_mnist.test.images.reshape(test_batch_size, n_step, n_input)
test_ys = fashion_mnist.test.labels

print('정확도:', sess.run(accuracy, feed_dict={X: test_xs, Y: test_ys}))


정확도: 0.8731
